In [1]:
import sys
sys.path.append('../src')

In [2]:
from spcqe.functions import individual_bases_3

In [2]:
import cvxpy as cp

ModuleNotFoundError: No module named 'cvxpy'

AttributeError: module 'pip' has no attribute 'list'

In [3]:
sys.path

['/Users/giray/Documents/github/spcqe/notebooks',
 '/opt/anaconda3/envs/spcqe/lib/python39.zip',
 '/opt/anaconda3/envs/spcqe/lib/python3.9',
 '/opt/anaconda3/envs/spcqe/lib/python3.9/lib-dynload',
 '',
 '/opt/anaconda3/envs/spcqe/lib/python3.9/site-packages',
 '../src']